In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
import itertools
from timeit import default_timer as timer
import scipy as sc
import pickle
import scipy.sparse as scs
from functools import lru_cache, cache
from joblib import Parallel, delayed


In [2]:
def boolean_combinations(n):
    return [
        *itertools.product(
            *[[0, 1, 2, 3] for _ in range(n)]
    )]

In [3]:
#inverted number of rishons of each link configuration
#0=Left, 1=Right, 2=Down

s0 = np.array([0,0,0])
s1 = np.array([1,1,0])
s2 = np.array([1,0,1])
s3 = np.array([0,1,1])

r = np.array([s0,s1,s2,s3]) #rishon register

In [4]:
def ten2four(n):
    n = n % (4294967296)
    s = np.base_repr(n,4)
    return np.array(list(s.zfill(16)),dtype=int)

In [5]:
gauge_indices = np.loadtxt("gauge_indices_4x4_sort1.txt").astype(int)
gauge_indices_base4 = np.array([ten2four(i) for i in gauge_indices])

In [7]:
def still_gauss(state):
    c = np.zeros((8,8388608),dtype=bool)

    c[0] = (r[state[:,12],1]+r[state[:,13],2])%2 == 0
    c[1] = (r[state[:,13],1]+r[state[:,14],2])%2 == 0
    c[2] = (r[state[:,14],1]+r[state[:,15],2])%2 == 0
    c[3] = (r[state[:,15],1]+r[state[:,12],2])%2 == 0

    c[4] = (r[state[:,0],0]) == 0
    c[5] = (r[state[:,1],0]) == 0
    c[6] = (r[state[:,2],0]) == 0
    c[7] = (r[state[:,3],0]) == 0

    return np.all(c,axis=0)

In [9]:
cond = still_gauss(gauge_indices_base4)

In [14]:
np.savetxt("gauge_indices_red.txt", gauge_indices[cond])